In [92]:
import pandas as pd
import numpy as np

In [93]:
df = pd.DataFrame({"A": [1,2,3,4,5,'',''], "B": [1,2,np.nan,4,np.nan,np.nan,np.nan]})
df2 = pd.DataFrame({"A": [1,2,'',4,5,6,7], "B": [1234234,2234234324,3223423,4232323,523323,1212,np.nan]})
df3 = pd.DataFrame({"A": [1,2,3,4,'',6,7], "B": [1,2,3,4,5,6,7]})

In [94]:
df

,A,B
0,1,1.0
1,2,2.0
2,3,NaN
3,4,4.0
4,5,NaN
5,,NaN
6,,NaN


In [95]:
df2

,A,B
0,1,1.234234e+06
1,2,2.234234e+09
2,,3.223423e+06
3,4,4.232323e+06
4,5,5.233230e+05
5,6,1.212000e+03
6,7,NaN


In [96]:
df3

,A,B
0,1,1
1,2,2
2,3,3
3,4,4
4,,5
5,6,6
6,7,7


In [98]:
def non_or_empty(x):
    def bitwise_non_or_empty(x):
        boo = False
        if np.isnan(x): 
            boo = True
        if x == None:
            boo = True
        if x == '':
            boo = True
        return boo
    boo = x.apply(
        lambda x: bitwise_non_or_empty(x))
    return boo

In [79]:
def join_and_update(left, right, left_on, right_on,
                    left_update, right_update, joined_on=""):
    
    left = left.copy()
    right = right.copy()
    
    left[left_on] = left[left_on].replace('', np.nan)
    right[right_on] = right[right_on].replace('', np.nan)
    
    df_join = left.merge(right.dropna(), how="left",
                         left_on=left_on, right_on=right_on,
                         suffixes=['','___y'])
    
    check = (non_or_empty(
        df_join[left_update])) & (~non_or_empty(
        df_join[right_update+'___y']))
            
    
    if 'Joined_on' not in df_join.columns:
        df_join['Joined_on'] = np.nan
    
    df_join.loc[check, 'Joined_on'] = joined_on
    df_join.loc[check, left_update] = df_join[right_update + '___y']
    df_join = df_join[[i for i in df_join.columns if not i.endswith('___y')]]
    return df_join

In [80]:
df4 = (df.pipe(join_and_update,
        right = df2,
        left_on="A", right_on="A",
        left_update="B", right_update="B",
        joined_on="Joined")
     .pipe(join_and_update,
        right = df3,
        left_on="A", right_on="A",
        left_update="B", right_update="B",
        joined_on="Joined"))

In [81]:
df4

,A,B,Joined_on
0,1.0,1.0,NaN
1,2.0,2.0,NaN
2,3.0,3.0,Joined
3,4.0,4.0,NaN
4,5.0,523323.0,Joined
5,NaN,NaN,NaN
6,NaN,NaN,NaN


In [82]:
def check_threshold(x, y, threshold):
    x = x.copy()
    y = y.copy()
    x[abs(x-y)>threshold] = np.nan
    return x

In [90]:
df4['B'] = df4['B'].pipe(check_threshold, df4['A'], 4)
df4['Joined_on'] = df4['B'].pipe(check_threshold, df4['A'], 4)

In [91]:
df4

,A,B,Joined_on
0,1.0,1.0,1.0
1,2.0,2.0,2.0
2,3.0,3.0,3.0
3,4.0,4.0,4.0
4,5.0,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
